In [15]:
import pickle
import pandas as pd
from transformers import TextClassificationPipeline, DistilBertForSequenceClassification, DistilBertTokenizer
import ast
import torch
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
model = pickle.load(open('embedding_model.pkl', 'rb'))
pipe = TextClassificationPipeline(model=model, tokenizer=tokenizer, return_all_scores=True)
le = pickle.load(open('label_enc.pkl', 'rb'))

/opt/conda/lib/python3.8/site-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar funcionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [21]:
### Temporary code cells
comparison_data = pd.read_excel("DATA/EDP_PropertyType_ML_results_comparison_0403.xlsx", sheet_name = "EDP_PropertyType_ML_comparison")

/opt/conda/lib/python3.8/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [22]:
comparison_data.head()

,EDP_Property_type,Count,myData_Property_manual_selection,result_from_ML_0403_Ravi,columnC,comparison
0,"50% DC, 50% OFFICE",2.0,1 - Office,Office,Office,match
1,Academic/Office,1.0,1 - Office,Education,Office,not match
2,ADM,229.0,1 - Office,Office,Office,match
3,Admin,865.0,1 - Office,Office,Office,match
4,ADMIN,304.0,1 - Office,Office,Office,match


In [28]:
comparison_data[comparison_data['comparison']=='not match']

,EDP_Property_type,Count,myData_Property_manual_selection,result_from_ML_0403_Ravi,columnC,comparison
1,Academic/Office,1.0,1 - Office,Education,Office,not match
8,"administration, production area and warehouse",1.0,1 - Office,Industrial & Logistics,Office,not match
9,Administrative,22.0,1 - Office,Public Institution,Office,not match
18,Parking,1352.0,1 - Office,Special Purpose,Office,not match
28,Parking Garage,148.0,1 - Office,Special Purpose,Office,not match
...,...,...,...,...,...,...
465,"Laboratory, General",45.0,9 - Healthcare,Laboratory,Healthcare,not match
467,HOS,7.0,9 - Healthcare,Office,Healthcare,not match
473,Medical Office Space,59.0,9 - Healthcare,Office,Healthcare,not match
474,Office/Clinical,3.0,9 - Healthcare,Office,Healthcare,not match


In [29]:
comparison_data.shape

(824, 6)

In [30]:
accuracy = 1 - (171/824)
accuracy

0.7924757281553398

In [31]:
input_data = list(comparison_data[comparison_data['comparison']=='not match']['EDP_Property_type'])
input_data[1]

'administration, production area and warehouse'

In [34]:
def converter(label_list):
    new_list = []
    for val in label_list:
        new_dict = {}
        new_dict[le.inverse_transform([int(val['label'].split('_')[1])])[0]] = round(val['score']*100,2)
        new_list.append(new_dict)
    return new_list

In [36]:
multiple_predictions = comparison_data[comparison_data['comparison']=='not match']
multiple_predictions['all_predictions'] = multiple_predictions['EDP_Property_type'].apply(lambda x:converter(sorted(pipe(x)[0], key=lambda item: item["score"], reverse=True)[:5]))
multiple_predictions

<ipython-input-36-3b73a293886f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  multiple_predictions['all_predictions'] = multiple_predictions['EDP_Property_type'].apply(lambda x:converter(sorted(pipe(x)[0], key=lambda item: item["score"], reverse=True)[:5]))


,EDP_Property_type,Count,myData_Property_manual_selection,result_from_ML_0403_Ravi,columnC,comparison,all_predictions
1,Academic/Office,1.0,1 - Office,Education,Office,not match,"[{'Education': 37.48}, {'Office': 19.95}, {'Pu..."
8,"administration, production area and warehouse",1.0,1 - Office,Industrial & Logistics,Office,not match,"[{'Industrial & Logistics': 94.31}, {'Office':..."
9,Administrative,22.0,1 - Office,Public Institution,Office,not match,"[{'Public Institution': 50.33}, {'Office': 36...."
18,Parking,1352.0,1 - Office,Special Purpose,Office,not match,"[{'Special Purpose': 97.75}, {'Office': 0.31},..."
28,Parking Garage,148.0,1 - Office,Special Purpose,Office,not match,"[{'Special Purpose': 97.86}, {'Industrial & Lo..."
...,...,...,...,...,...,...,...
465,"Laboratory, General",45.0,9 - Healthcare,Laboratory,Healthcare,not match,"[{'Laboratory': 92.69}, {'Industrial & Logisti..."
467,HOS,7.0,9 - Healthcare,Office,Healthcare,not match,"[{'Office': 70.87}, {'Special Purpose': 6.3}, ..."
473,Medical Office Space,59.0,9 - Healthcare,Office,Healthcare,not match,"[{'Office': 97.33}, {'Special Purpose': 0.42},..."
474,Office/Clinical,3.0,9 - Healthcare,Office,Healthcare,not match,"[{'Office': 97.77}, {'Public Institution': 0.3..."


In [37]:
multiple_predictions = multiple_predictions.explode('all_predictions')
multiple_predictions.head()

,EDP_Property_type,Count,myData_Property_manual_selection,result_from_ML_0403_Ravi,columnC,comparison,all_predictions
1,Academic/Office,1.0,1 - Office,Education,Office,not match,{'Education': 37.48}
1,Academic/Office,1.0,1 - Office,Education,Office,not match,{'Office': 19.95}
1,Academic/Office,1.0,1 - Office,Education,Office,not match,{'Public Institution': 10.63}
1,Academic/Office,1.0,1 - Office,Education,Office,not match,{'Senior Housing': 9.62}
1,Academic/Office,1.0,1 - Office,Education,Office,not match,{'Laboratory': 5.52}


In [38]:
multiple_predictions['predicted_category'] = multiple_predictions['all_predictions'].apply(lambda x:x.keys())
multiple_predictions['prediction_probability'] = multiple_predictions['all_predictions'].apply(lambda x:x.values())
multiple_predictions.head()

,EDP_Property_type,Count,myData_Property_manual_selection,result_from_ML_0403_Ravi,columnC,comparison,all_predictions,predicted_category,prediction_probability
1,Academic/Office,1.0,1 - Office,Education,Office,not match,{'Education': 37.48},(Education),(37.48)
1,Academic/Office,1.0,1 - Office,Education,Office,not match,{'Office': 19.95},(Office),(19.95)
1,Academic/Office,1.0,1 - Office,Education,Office,not match,{'Public Institution': 10.63},(Public Institution),(10.63)
1,Academic/Office,1.0,1 - Office,Education,Office,not match,{'Senior Housing': 9.62},(Senior Housing),(9.62)
1,Academic/Office,1.0,1 - Office,Education,Office,not match,{'Laboratory': 5.52},(Laboratory),(5.52)


In [39]:
multiple_predictions[['EDP_Property_type', 'myData_Property_manual_selection','result_from_ML_0403_Ravi','comparison','predicted_category','prediction_probability']].to_excel("EDP_Failed_top_5_0505.xlsx", index=False)

In [ ]:
### Temporary cells end, uncomment next one for pipeline

In [ ]:
input_data = list(pd.read_csv("DATA/EDP_Data_040323.csv").iloc[:,0])
input_data[1]

In [10]:
results = []
for val in input_data:
    try:
        sorted_labels = le.inverse_transform([int(sorted(pipe(val)[0], key=lambda item: item["score"], reverse=True)[0]['label'].split("_")[1])])[0]
        results.append(sorted_labels)
    except Exception as e:
        print(val)
        results.append("Not found")

nan


In [11]:
len(results)

824

In [13]:
len(input_data)

824

In [14]:
new_data_set = pd.DataFrame([input_data,results]).T.rename(columns={0:"Input_data",1:"Category"})
new_data_set.to_csv("DATA/EDP_data_results_040323.csv", index=False)